In [2]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("train.csv", index_col = 0)
test_dataset = pd.read_csv("test.csv", index_col = 0)

In [27]:
# print(dataset.head())
print(test_dataset.describe())
print(test_dataset.isna().sum(axis = 0))

           Pclass         Age       SibSp       Parch        Fare
count  418.000000  418.000000  418.000000  418.000000  417.000000
mean     2.265550   29.908293    0.447368    0.392344   35.627188
std      0.841838   13.276662    0.896760    0.981429   55.907576
min      1.000000    0.170000    0.000000    0.000000    0.000000
25%      1.000000   22.000000    0.000000    0.000000    7.895800
50%      3.000000   29.328244    0.000000    0.000000   14.454200
75%      3.000000   36.375000    1.000000    0.000000   31.500000
max      3.000000   76.000000    8.000000    9.000000  512.329200
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        1
Cabin       0
Embarked    0
dtype: int64


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

np.random.seed(100)

age_regr_maxdepth = 4

X_train, y_train = dataset.loc[:, dataset.columns != 'Survived'], dataset.Survived
X_test = test_dataset.copy()

X_train.Cabin = X_train.Cabin.apply(lambda x: "unk" if x is np.nan else x.split()[0][0])
X_test.Cabin = X_test.Cabin.apply(lambda x: "unk" if x is np.nan else x.split()[0][0])

embarked_class = X_train.groupby("Embarked").count()['Pclass'].idxmax()
X_train.Embarked = X_train.Embarked.apply(lambda x: embarked_class if x is np.nan else x)
X_test.Embarked = X_test.Embarked.apply(lambda x: embarked_class if x is np.nan else x)

X_test.Fare = X_test.Fare.fillna(np.nanmedian(X_test.Fare))

title_train = X_train.Name.apply(lambda x: x.split(",")[1].split()[0])
title_test = X_test.Name.apply(lambda x: x.split(",")[1].split()[0])

X_age_train = X_train.loc[:, ['Pclass', 'Sex', 'Fare']]
X_age_train['Title'] = title_train
X_age_test = X_test.loc[:, ['Pclass', 'Sex', 'Fare']]
X_age_test['Title'] = title_test
y_age_train = X_train.Age
y_age_test = X_test.Age

ord_enc1 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = np.nan)
ord_enc1.fit(X_age_train.loc[:, ['Pclass']])
X_age_train.Pclass = ord_enc1.transform(X_age_train.loc[:, ['Pclass']])
X_age_test.Pclass = ord_enc1.transform(X_age_test.loc[:, ['Pclass']])

oh_enc1 = OneHotEncoder(handle_unknown='ignore')
oh_enc1.fit(X_age_train.loc[:, ['Sex', 'Title']])
X_age_train[oh_enc1.get_feature_names_out()] = oh_enc1.transform(X_age_train.loc[:, ['Sex', 'Title']]).todense()
X_age_test[oh_enc1.get_feature_names_out()] = oh_enc1.transform(X_age_test.loc[:, ['Sex', 'Title']]).todense()

X_age_train = X_age_train.drop(['Sex', 'Title'], axis=1)
X_age_test = X_age_test.drop(['Sex', 'Title'], axis=1)

age_regr = DecisionTreeRegressor(max_depth=age_regr_maxdepth)

# cv_score_sim = 0
# r2_score_sim = 0
# 
# n = 50
# for i in range(n):
    # cv_score_sim += cross_val_score(age_regr, X_age_train, y_age_train, cv=5).mean()
    # age_regr.fit(X_age_train, y_age_train) 
    # r2_score_sim += r2_score(y_age_val, age_regr.predict(X_age_val))

age_regr.fit(X_age_train.loc[y_age_train.notna(), :], y_age_train[y_age_train.notna()])
X_train.Age[X_train.Age.isna()] = age_regr.predict(X_age_train.loc[y_age_train.isna(), :])
X_test.Age[X_test.Age.isna()] = age_regr.predict(X_age_test.loc[y_age_test.isna(), :])

X_train = X_train.drop(["Name", "Ticket"], axis = 1)
X_test = X_test.drop(["Name", "Ticket"], axis = 1)

oh_enc2 = OneHotEncoder(handle_unknown='ignore')
oh_enc2.fit(X_train.loc[:, ['Sex', 'Cabin', 'Embarked']])
X_train[oh_enc2.get_feature_names_out()] = oh_enc2.transform(X_train.loc[:, ['Sex', 'Cabin', 'Embarked']]).todense()
X_test[oh_enc2.get_feature_names_out()] = oh_enc2.transform(X_test.loc[:, ['Sex', 'Cabin', 'Embarked']]).todense()

X_train = X_train.drop(['Sex', 'Cabin', 'Embarked'], axis = 1)
X_test = X_test.drop(['Sex', 'Cabin', 'Embarked'], axis = 1)


poly = PolynomialFeatures()
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

svm = SVC(C=0.5)
svm.fit(X_train, y_train)
svm.predict(X_test)



C:\Users\yunwa\AppData\Local\Temp\ipykernel_32940\1356707525.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.Cabin = X_train.Cabin.apply(lambda x: "unk" if x is np.nan else x.split()[0][0])
C:\Users\yunwa\AppData\Local\Temp\ipykernel_32940\1356707525.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.Embarked = X_train.Embarked.apply(lambda x: embarked_class if x is np.nan else x)
C:\Users\yunwa\AppData\Local\Temp\ipykernel_32940\1356707525.py:61: SettingWithCopyWarning: 
A value is t

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC



age_regr_maxdepth = 4
rfr_estimators = 100
rfr_max_depth = 4
rfr_a = 0.01

X_train, X_val, y_train, y_val = train_test_split(dataset.loc[:, dataset.columns != 'Survived'], dataset.Survived, test_size = 0.2)

X_train.Cabin = X_train.Cabin.apply(lambda x: "unk" if x is np.nan else x.split()[0][0])
X_val.Cabin = X_val.Cabin.apply(lambda x: "unk" if x is np.nan else x.split()[0][0])

embarked_class = X_train.groupby("Embarked").count()['Pclass'].idxmax()
X_train.Embarked = X_train.Embarked.apply(lambda x: embarked_class if x is np.nan else x)
X_val.Embarked = X_val.Embarked.apply(lambda x: embarked_class if x is np.nan else x)

title_train = X_train.Name.apply(lambda x: x.split(",")[1].split()[0])
title_val = X_val.Name.apply(lambda x: x.split(",")[1].split()[0])

X_age_train = X_train.loc[:, ['Pclass', 'Sex', 'Fare']]
X_age_train['Title'] = title_train
X_age_val = X_val.loc[:, ['Pclass', 'Sex', 'Fare']]
X_age_val['Title'] = title_val
y_age_train = X_train.Age
y_age_val = X_val.Age

ord_enc1 = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value = np.nan)
ord_enc1.fit(X_age_train.loc[:, ['Pclass']])
X_age_train.Pclass = ord_enc1.transform(X_age_train.loc[:, ['Pclass']])
X_age_val.Pclass = ord_enc1.transform(X_age_val.loc[:, ['Pclass']])

oh_enc1 = OneHotEncoder(handle_unknown='ignore')
oh_enc1.fit(X_age_train.loc[:, ['Sex', 'Title']])
X_age_train[oh_enc1.get_feature_names_out()] = oh_enc1.transform(X_age_train.loc[:, ['Sex', 'Title']]).todense()
X_age_val[oh_enc1.get_feature_names_out()] = oh_enc1.transform(X_age_val.loc[:, ['Sex', 'Title']]).todense()

X_age_train = X_age_train.drop(['Sex', 'Title'], axis=1)
X_age_val = X_age_val.drop(['Sex', 'Title'], axis=1)

age_regr = DecisionTreeRegressor(max_depth=age_regr_maxdepth)

# cv_score_sim = 0
# r2_score_sim = 0
# 
# n = 50
# for i in range(n):
    # cv_score_sim += cross_val_score(age_regr, X_age_train, y_age_train, cv=5).mean()
    # age_regr.fit(X_age_train, y_age_train) 
    # r2_score_sim += r2_score(y_age_val, age_regr.predict(X_age_val))

age_regr.fit(X_age_train.loc[y_age_train.notna(), :], y_age_train[y_age_train.notna()])
X_train.Age[X_train.Age.isna()] = age_regr.predict(X_age_train.loc[y_age_train.isna(), :])
X_val.Age[X_val.Age.isna()] = age_regr.predict(X_age_val.loc[y_age_val.isna(), :])

X_train = X_train.drop(["Name", "Ticket"], axis = 1)
X_val = X_val.drop(["Name", "Ticket"], axis = 1)

oh_enc2 = OneHotEncoder(handle_unknown='ignore')
oh_enc2.fit(X_train.loc[:, ['Sex', 'Cabin', 'Embarked']])
X_train[oh_enc2.get_feature_names_out()] = oh_enc2.transform(X_train.loc[:, ['Sex', 'Cabin', 'Embarked']]).todense()
X_val[oh_enc2.get_feature_names_out()] = oh_enc2.transform(X_val.loc[:, ['Sex', 'Cabin', 'Embarked']]).todense()

X_train = X_train.drop(['Sex', 'Cabin', 'Embarked'], axis = 1)
X_val = X_val.drop(['Sex', 'Cabin', 'Embarked'], axis = 1)


poly = PolynomialFeatures()
X_train = poly.fit_transform(X_train)
X_val = poly.transform(X_val)

# rfr = RandomForestClassifier(n_estimators=rfr_estimators, max_depth=rfr_max_depth, ccp_alpha=rfr_a)
# rfr.fit(X_train, y_train)
# accuracy_score(y_val, rfr.predict(X_val))

# param_grid = [
#     {'n_estimators': [1, 10, 50, 100, 200, 500],
#      'max_depth': [1, 3, 5, 7, 10],
#      'ccp_alpha': [0, 0.01, 0.05, 0.01, 0.2, 0.5],
#      'max_features': ['sqrt', 'auto', 'log2']}
# ]

# # gscv = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, n_jobs = -1)
# # gscv.fit(X_train, y_train)

param_grid = [
    {'C': [0.3, 0.4, 0.045, 0.055, 0.5, 0.6, 0.7, 0.8, 1, 2, 3]}
]

gscv = GridSearchCV(SVC(), param_grid=param_grid, n_jobs = -1)
gscv.fit(X_train, y_train)

C:\Users\yunwa\AppData\Local\Temp\ipykernel_32940\4251634829.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.Age[X_train.Age.isna()] = age_regr.predict(X_age_train.loc[y_age_train.isna(), :])
C:\Users\yunwa\AppData\Local\Temp\ipykernel_32940\4251634829.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.Age[X_val.Age.isna()] = age_regr.predict(X_age_val.loc[y_age_val.isna(), :])


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid=[{'C': [0.3, 0.4, 0.045, 0.055, 0.5, 0.6, 0.7, 0.8, 1,
                                2, 3]}])

In [10]:
gscv.best_estimator_

SVC(C=0.5)